# 【語音辨識 - Whisper】 準確與否需要有一把 📏尺來衡量


前面我們介紹了幾個關於Whisper的基本概念，這裡附上 [🚀傳送門](https://vocus.cc/article/644526c8fd89780001ffdd9f) ，歡迎好好閱讀一番，但我們除了學會如何用語音辨識的工具之外，「準確率」對我們來說也是一個非常重要的一環，但我們究竟應該要如何評估所謂的準確率呢？ 不知道沒關係，當您看完這個篇章就能夠學會如何計算文字的「字元錯誤率」、「字詞錯誤率」...，非常值得您細細品嘗與學習，就讓我們往下一步步的完成評估準確率的程序吧！

這次的評估工具我們會使用jiwer這一套來進行說明，它支援了多種的計算方式，包括： WER、CER、MER...等，那這些計算方式各有什麼不同呢？ 就讓我們繼續看下去吧！

In [ ]:
#程式修改來自 https://github.com/weihanchen/google-colab-python-learn/blob/main/jupyter-examples/whisper/whisper_acc.ipynb

## 安裝套件

In [1]:
# 錯誤率計算工具
!pip install jiwer

# 移除掉與語音辨識套件相同名稱的套件
# !pip uninstall whisper

# 語音辨識ASR
#!pip install -U openai-whisper

# Hugging Face資料集函式庫
#!pip install datasets

# 斷詞器
#!pip install jiaba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.6 MB/s eta 0:00:00


## 有哪些不同的計算方式呢？

### 以「詞」為單位進行計算

我們先來看看詞的計算結果如下：

In [3]:
import jiwer

reference = "今天 天氣 很好 嗎"
hypothesis = "今天 天氣 很 好 啊"

out = jiwer.process_words(reference, hypothesis)
print(jiwer.visualize_alignment(out))

sentence 1
REF: 今天 天氣 * 很好 嗎
HYP: 今天 天氣 很  好 啊
           I  S S

number of sentences: 1
substitutions=2 deletions=0 insertions=1 hits=2

mer=60.00%
wil=80.00%
wip=20.00%
wer=75.00%



#### 詞錯誤率 Word Error Rate(WER)
WER是以「詞」為單位進行計算，它用來衡量句子中有多少詞彙需要進行修改才能和正確答案一樣。

```bash
公式: (S + D + I) / (H + S + D)
計算過程: (2 + 0 + 1) / (2 + 2 + 0)
3 / 4 ≈ 75%。
```

💡 既然是以`詞`為單位的話，那麼我們的答案與辨識結果請先進行斷詞(通常用空白隔開)， 標點符號也是考量的因素之一喔。

#### 平均錯誤率 Mean Error Rate(MER)
這項指標與WER主要差別在於分母的部分尚未將`Insertion`給考量進來計算，因為它衡量的不僅是詞彙層級，而是句子層級，因此會更加全面。

```bash
公式： (S + D + I) / (H + S + D + I)
計算過程： (2 + 0 + 1) / (2 + 2 + 0 + 1)

3 / 5 ≈ 60%
```

#### 詞保留率 Word Information Preservation(WIP)
這項指標主要在評估我們的辨識結果究竟有多少比例的字詞是一模一樣完全正確的。

```bash
num_rf_words = 正確答案字詞數 = 4
num_hp_words = 辨識結果字詞數 = 5
公式： (H / num_rf_words) * (H / num_hp_words)
計算過程: (2 / 4) * (2 / 5)
0.5 * 0.4 ≈ 20%
```
#### 詞漏失率 Word Information Lost(WIL)
既然有詞的保留率，那麼相反的就是漏失率，因此上述的結果得出之後，用1減去保留率就是漏失率，可以粗略的評估總共漏失了多少比率。
```bash
公式: 1 - wip
1 - 0.2 ≈ 80%
```

## 以「字元」為單位進行計算

### 字元錯誤率 Character Error Rate(CER)
CER是以「字元」為單位進行計算，底下的例子以「字元」為單位會發現有1個substitution，因此總共7個字元錯了1個等於：

```
1 / 7 = 14.28%
```

💡 既然是以`字元`為單位的話，那麼我們的答案與辨識結果請將空白給去除， 才不會也被計算進去喔， 甚至標點符號...等都是考量的因素之一。

In [ ]:
#連到到自己的google drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [ ]:
import re
def remove_repeated_words(text):
    pattern = r'(\w{2,100})\1'
    while True:
        new_text = re.sub(pattern, r'\1', text)
        if new_text == text: break
        text = new_text
    return text

# new_transcript = []
# for transcript in transcripts:
#     new_transcript.append(remove_repeated_words(transcript))

In [ ]:
import jiwer
#seiching
with open("566_323.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/v2/20221025_566_correctc.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"
#hypothesis=remove_repeated_words(hypothesis)
error = jiwer.cer(reference, hypothesis)
print(error)
output = jiwer.process_characters(reference, hypothesis)
#print(jiwer.visualize_alignment(output))


0.412135442412259


In [ ]:
reference


'六十六次市政會議首先我們進行獻獎請市長站到臺前接受獻獎二零二二國家卓越建設獎暨全球卓越建設獎臺南市供流榮獲三十六座獎座得獎數全國第一今日共有三位人員代表獎獻獎首先是公務局共計十二座獎項新宜區公三公園新闢工程南區公六一公園闢建工程鹽水區公園新闢工程南寧高中校舍重建工程利其馬颱風道路邊坡復建工程臺南水道道路拓寬工程公六二及公六五闢建工程鹽水水岸公園建制工程四道一七二線至白河區拓寬工程中西區合維路五段延伸工程南區公六一公園闢建工程施工品質金質獎由林志穎科長獻獎鹽水區公園新闢工程施工品質金質獎由歐好義正工程師獻獎南寧高中校舍重建工程由趙嘉賢副工程師獻獎利其瑪颱風道路復建工程由張國元副大隊長獻獎臺南水道道路斷工程由謝銘宏股長獻獎二級攻六五闢建工程由戴仲青副工程師獻獎鹽水水岸公園建制工程由許恆獻獎四道二線白河區拓寬工程由謝明宏股長獻獎中西區合圍路五段鹽水水岸工程由新建科黃俊豪正工程師獻獎臺南市公園特色遊戲場規劃設計案由公園科游明亮科員獻獎區新營醫院北門分院整建工程由建工科黃玉成正工程師獻獎請市長及獻獎人員留步公務局各位獻獎代表請至臺上合影來我們看旁邊的近一點麻煩接著是水利局共計八座獎項首先臺南市虎尾寮汙水下水道管線工程施工品質類金質獎由韓龍華局長代表獻獎鹽水排水治理工程由曾俊傑幫工程師獻獎石仔瀨排水抽水更新工程由石群龍先生獻獎龜子港排水護腕治理工程由陳建仁副工程師獻獎仁德區汙水下水道工程由蔡英明科長獻獎海東抽水站新建工程由邱建志辦工程師獻獎麻豆溼地重劃抽水站新建工程由林玉年科長獻獎坊溝尾溪水質淨化廠由郭伯渝幫工程師獻獎請市長及獻獎人員留步水利局各位獻獎代表請至台上合影好請局長各位獻獎代表回座教育局共計六座獎項九分子國民中小學新建工程規劃設計卓越獎由康進園校長及鄭心輝局長共同代表獻獎善化區小興國小遷校新建工程由謝慶黃校長獻獎歸仁國民小學重建工程由羅俊男校長獻獎智開實驗小學校舍重建工程由王念鄉校長獻獎鹽行中學新社校工程環境文化類由林建佑校長獻獎鹽行中學新社校工程施工平穩定由胡振良主任獻獎請教育局長官留步至臺上合影請局長及各位獻獎代表回座地政局共計四座獎項鹽行國中附近區段工程施工品之類金質獎由陳淑美局長代表獻獎永康六甲頂醫療專區溼地重劃工程規劃設計由邱龍捷科長獻獎安南區怡中溼地重劃工程規劃設計金質獎由戴為良股長獻獎南區儀中溼地重劃工程規劃設計金值獎由代為良股長獻獎喜樹灣裡溼地重劃滯

In [ ]:
import jiwer
#openai
with open("566.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/v2/20221025_566_correctc.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"
#hypothesis=remove_repeated_words(hypothesis)
error = jiwer.cer(reference, hypothesis)
print(error)
output = jiwer.process_characters(reference, hypothesis)
#print(jiwer.visualize_alignment(output))

0.5098863074641621


In [ ]:
import jiwer
#openai
with open("609.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/v2/20230829_609_correctc.txt","r",encoding="utf-8") as correctedasr:

  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"
#hypothesis=remove_repeated_words(hypothesis)
error = jiwer.cer(reference, hypothesis)
print(error)
output = jiwer.process_characters(reference, hypothesis)
#print(jiwer.visualize_alignment(output))

0.5004908912403185


In [ ]:
import jiwer
#openai
with open("626.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/v2/20231225_626_correctc.txt","r",encoding="utf-8") as correctedasr:

  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"
#hypothesis=remove_repeated_words(hypothesis)
error = jiwer.cer(reference, hypothesis)
print(error)
output = jiwer.process_characters(reference, hypothesis)
#print(jiwer.visualize_alignment(output))

0.5162868042648929


In [ ]:
import jiwer
#seiching
with open("609_323.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/v2/20230829_609_correctc.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"
#hypothesis=remove_repeated_words(hypothesis)
error = jiwer.cer(reference, hypothesis)
print(error)
output = jiwer.process_characters(reference, hypothesis)
#print(jiwer.visualize_alignment(output))

In [ ]:
hypothesis

'在六十六次市政會議之前我們進行獻獎請市長站大台前接受獧獎項沿水水岸公園建置工程，世道一切惡限制白河區拓寬工程，中西區河圍路五段延�富工程式獻獎臺南市公園特色遊戲場規劃設計案由公園科由明亮科園獻獎區域也信義醫院北門分院整建工程由建工科黃裕誠正工程式獻獎請市長及獻獎人員留步公務計八座獎獎水利局各位現長代表回座第一正局共計四座獎獎延航國中附近區大股分或工程是公平之類金職獎由陳書美局長及各位現場代表回座來我們麻煩看前方鏡頭由吳宇芝處長獻獎蔣永康區青春社會住宅類伸展請市長及局長留步，科黨單位共同合影並請各位新黨代表近數字台前那我們請局處長官以及校長站在就是我們第一排合影處那我們各位新黨代表在第二排及第三排因為人數較多麻煩第二排及第三排的長官請偬身包括公務局、包括水利局、包括交通局、包括地政局以及包括都發局、包括教育局、包括文化局以及相關各位學校以及機關的長官、校長、各位好好我想公眾公眾基礎建設規劃完善是一個城市進步的象徵那麼城市因為建築而偉大那民眾因為建築能夠安居能夠樂業偉大的城市是落實宜居以及合宜居住環境那由我們台南市這一次在今年度在國家卓越建設獎中間大放異彩總共獲得三十五座獎杯以及全球卓越建設獎銀獎一座那這個是非常非常高的榮譽榮耀也是非常高的榮譽那這些建築都是一個公眾工程品質的一個表徵也是一個城市在讓城縣在市民朋友還有外來朋友面前一個讓人家覺得這個城市進步的一個表徵那非常謝謝那尤其我們臺南市立圖書館總館新建總館也獲得了還有我們水交涉也獲得了全球逐月獎銀獎一座這點是非常非常不容易的在全球的激烈評比中間雖然不是金獎，但是我們覺得來城市營造以及城市的進步能夠更為彰顯再次感謝也再次祝福我們所有得獎的各個單位、各個機關謝謝各位，市民以各位為榮，謝謝，好再來。確認上次會議議事錄請問各位同仁對於上次會議議事錄有沒有補充�沒有的話就通過報告事項報告事項報告事項一水利局呈報本是沿海地區採部落防護治水黃紅工法對策及成效以北門區及區域為例的報告請楊金豪科長報告時間五分鐘好科長你可以開始了趕趕退朝的影響甚至如果遇到一些颱風或地氣壓也會拉高整個朝位加劇的整個社區排放的困難所以依據網路降下豪大雨這會造成一些積煙水災情我以一百零五年沒積颱風為例在二十四小時降下三百毫米的這個降雨事件就煙水面積就有三百四十公頃主要的煙水範圍包含了北門的井湖、玉港、雪甲的星座端等地勢比較低溫的地區那我們進一步來探討這些社

In [ ]:
import jiwer
with open("seihing20221025 臺南市政府第566次市政會議.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/20221025-臺南市政府第566次市政會議_正確版.docx.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"

#output = jiwer.process_characters(reference, hypothesis)
#print(jiwer.visualize_alignment(output))
error = jiwer.cer(reference, hypothesis)
print(error)


0.39475232380280034


In [ ]:
import jiwer
with open("openai20230829台南市政府第609次市政會議 直播.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/202310829 台南市政府 第609次市政會議 正確版.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"

#output = jiwer.process_characters(reference, hypothesis)
#print(jiwer.visualize_alignment(output))
error = jiwer.cer(reference, hypothesis)
print(error)

0.2962521125360374


In [ ]:
import jiwer
with open("seiching20230829台南市政府第609次市政會議 直播.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/202310829 台南市政府 第609次市政會議 正確版.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"

error = jiwer.cer(reference, hypothesis)
print(error)

0.3848295059151009


In [ ]:
import jiwer
with open("openai20231225 台南市政府 第626次市政會議 直播.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/20231225 台南市政府 第626次市政會議 正確版.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"

error = jiwer.cer(reference, hypothesis)
print(error)

0.36810352365130716


In [ ]:
import jiwer
with open("seiching20231225 台南市政府 第626次市政會議 直播.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/20231225 台南市政府 第626次市政會議 正確版.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"

error = jiwer.cer(reference, hypothesis)
print(error)

0.37789630147766023


In [ ]:
import jiwer
with open("openai20240130 台南市政府 第631次市政會議 直播.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/20240130 台南市政府 第631次市政會議 正確版.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"

error = jiwer.cer(reference, hypothesis)
print(error)

0.3496987951807229


In [ ]:
import jiwer
with open("seiching20240130 台南市政府 第631次市政會議 直播.txt","r",encoding="utf-8") as openaiasr:
  hypothesis=openaiasr.read()
with open("corrected/20240130 台南市政府 第631次市政會議 正確版.txt","r",encoding="utf-8") as correctedasr:
  reference=correctedasr.read()

#reference = "今天天氣很好嗎"
#hypothesis = "今天天氣很好啊"

error = jiwer.cer(reference, hypothesis)
print(error)

0.35873493975903614
